# XLOT 1 

In [2]:
from pathlib import Path
from datetime import datetime, timedelta
import re


def convert_delft_las_to_txt_simple(las_path: Path, out_path: Path):
    """
    Convert Delft LAS file to a simple TXT with:
      Datetime, TIME [s], APQJ [bar], CPTV [cm3], CPTVC1 [cm3],
      PTVC1 [cm3], RTDQJ [degC], SPEP [psi]

    - Datetime is reconstructed from TLAB. (or DATE. fallback) + TIME seconds.
    - All values are kept in the units given in the LAS file.
    """
    las_path = Path(las_path)
    out_path = Path(out_path)

    with open(las_path, "r", errors="ignore") as f:
        lines = f.readlines()

    # ----------------------------------------------------
    # 1) Extract absolute start time t0 from TLAB. (preferred) or DATE.
    # ----------------------------------------------------
    t0 = None
    for line in lines:
        if line.strip().startswith("TLAB."):
            # Example: "TLAB.       16:00 01-NOV-2025  : Time Logger At Bottom"
            m = re.search(r"TLAB\.\s+([0-9:]+)\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
            if m:
                time_str, date_str = m.groups()
                t0 = datetime.strptime(f"{time_str} {date_str}", "%H:%M %d-%b-%Y")
                break

    if t0 is None:
        for line in lines:
            if line.strip().startswith("DATE."):
                # Example: "DATE.       01-NOV-2025 : Log Date"
                m = re.search(r"DATE\.\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
                if m:
                    date_str = m.group(1)
                    t0 = datetime.strptime(f"00:00 {date_str}", "%H:%M %d-%b-%Y")
                    break

    if t0 is None:
        # Very defensive fallback
        t0 = datetime(2025, 1, 1)

    # ----------------------------------------------------
    # 2) Locate the ~A block and parse curve names
    # ----------------------------------------------------
    a_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("~A"):
            a_idx = i
            break
    if a_idx is None:
        raise RuntimeError("No ~A block found in LAS file.")

    header_line = lines[a_idx].strip()
    curve_names = header_line.split()[1:]  # skip "~A"
    col_idx = {name: j for j, name in enumerate(curve_names)}

    # Ensure required curves exist
    required_curves = ["TIME", "APQJ", "CPTV", "CPTVC1", "PTVC1", "RTDQJ", "SPEP"]
    for rc in required_curves:
        if rc not in col_idx:
            raise RuntimeError(f"Required curve {rc} not found in LAS file.")

    ncols = len(curve_names)

    # ----------------------------------------------------
    # 3) Write TXT: datetime + named columns
    # ----------------------------------------------------
    with open(out_path, "w", encoding="utf-8") as out:
        # Single header row with clear names + units
        out.write(
            "Datetime"
            "\tTIME [s]"
            "\tAPQJ [bar]"
            "\tCPTV [cm3]"
            "\tCPTVC1 [cm3]"
            "\tPTVC1 [cm3]"
            "\tRTDQJ [degC]"
            "\tSPEP [psi]\n"
        )

        # Data lines
        for line in lines[a_idx + 1:]:
            if not line.strip():
                continue

            parts = line.strip().split()
            if len(parts) != ncols:
                continue  # malformed / end-of-data

            try:
                time_s = float(parts[col_idx["TIME"]])
                apqj   = float(parts[col_idx["APQJ"]])
                cptv   = float(parts[col_idx["CPTV"]])
                cptvc1 = float(parts[col_idx["CPTVC1"]])
                ptvc1  = float(parts[col_idx["PTVC1"]])
                rtdqj  = float(parts[col_idx["RTDQJ"]])
                spep   = float(parts[col_idx["SPEP"]])
            except ValueError:
                # skip lines that can't be parsed numerically
                continue

            # Build datetime from t0 + TIME [s]
            timestamp = t0 + timedelta(seconds=time_s)
            dt_str = timestamp.strftime("%d-%m-%Y %H:%M:%S")

            out.write(
                f"{dt_str}"
                f"\t{time_s:.3f}"
                f"\t{apqj:.4f}"
                f"\t{cptv:.3f}"
                f"\t{cptvc1:.3f}"
                f"\t{ptvc1:.3f}"
                f"\t{rtdqj:.3f}"
                f"\t{spep:.3f}\n"
            )

    print(f"Wrote TXT with datetime + named columns to: {out_path}")

# Example usage:


las_file = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_S2_7in_WL_run31_01Nov2025_RCX_MICROFRAC_2267_4M.las")
out_txt  = Path("TU_DELFT_GT02_MICROFRAC_converted_simple.txt")
convert_delft_las_to_txt_simple(las_file, out_txt)
print("Written:", out_txt)

Wrote TXT with datetime + named columns to: TU_DELFT_GT02_MICROFRAC_converted_simple.txt
Written: TU_DELFT_GT02_MICROFRAC_converted_simple.txt


# XLOT 2

In [1]:
from pathlib import Path
from datetime import datetime, timedelta
import re


def convert_delft_las_to_txt_simple(las_path: Path, out_path: Path):
    """
    Convert Delft LAS file to a simple TXT with:
      Datetime, TIME [s], APQJ [bar], CPTV [cm3], CPTVC1 [cm3],
      PTVC1 [cm3], RTDQJ [degC], SPEP [psi]

    - Datetime is reconstructed from TLAB. (or DATE. fallback) + TIME seconds.
    - All values are kept in the units given in the LAS file.
    """
    las_path = Path(las_path)
    out_path = Path(out_path)

    with open(las_path, "r", errors="ignore") as f:
        lines = f.readlines()

    # ----------------------------------------------------
    # 1) Extract absolute start time t0 from TLAB. (preferred) or DATE.
    # ----------------------------------------------------
    t0 = None
    for line in lines:
        if line.strip().startswith("TLAB."):
            # Example: "TLAB.       16:00 01-NOV-2025  : Time Logger At Bottom"
            m = re.search(r"TLAB\.\s+([0-9:]+)\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
            if m:
                time_str, date_str = m.groups()
                t0 = datetime.strptime(f"{time_str} {date_str}", "%H:%M %d-%b-%Y")
                break

    if t0 is None:
        for line in lines:
            if line.strip().startswith("DATE."):
                # Example: "DATE.       01-NOV-2025 : Log Date"
                m = re.search(r"DATE\.\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
                if m:
                    date_str = m.group(1)
                    t0 = datetime.strptime(f"00:00 {date_str}", "%H:%M %d-%b-%Y")
                    break

    if t0 is None:
        # Very defensive fallback
        t0 = datetime(2025, 1, 1)

    # ----------------------------------------------------
    # 2) Locate the ~A block and parse curve names
    # ----------------------------------------------------
    a_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("~A"):
            a_idx = i
            break
    if a_idx is None:
        raise RuntimeError("No ~A block found in LAS file.")

    header_line = lines[a_idx].strip()
    curve_names = header_line.split()[1:]  # skip "~A"
    col_idx = {name: j for j, name in enumerate(curve_names)}

    # Ensure required curves exist
    required_curves = ["TIME", "APQJ", "CPTV", "CPTVC1", "PTVC1", "RTDQJ", "SPEP"]
    for rc in required_curves:
        if rc not in col_idx:
            raise RuntimeError(f"Required curve {rc} not found in LAS file.")

    ncols = len(curve_names)

    # ----------------------------------------------------
    # 3) Write TXT: datetime + named columns
    # ----------------------------------------------------
    with open(out_path, "w", encoding="utf-8") as out:
        # Single header row with clear names + units
        out.write(
            "Datetime"
            "\tTIME [s]"
            "\tAPQJ [bar]"
            "\tCPTV [cm3]"
            "\tCPTVC1 [cm3]"
            "\tPTVC1 [cm3]"
            "\tRTDQJ [degC]"
            "\tSPEP [psi]\n"
        )

        # Data lines
        for line in lines[a_idx + 1:]:
            if not line.strip():
                continue

            parts = line.strip().split()
            if len(parts) != ncols:
                continue  # malformed / end-of-data

            try:
                time_s = float(parts[col_idx["TIME"]])
                apqj   = float(parts[col_idx["APQJ"]])
                cptv   = float(parts[col_idx["CPTV"]])
                cptvc1 = float(parts[col_idx["CPTVC1"]])
                ptvc1  = float(parts[col_idx["PTVC1"]])
                rtdqj  = float(parts[col_idx["RTDQJ"]])
                spep   = float(parts[col_idx["SPEP"]])
            except ValueError:
                # skip lines that can't be parsed numerically
                continue

            # Build datetime from t0 + TIME [s]
            timestamp = t0 + timedelta(seconds=time_s)
            dt_str = timestamp.strftime("%d-%m-%Y %H:%M:%S")

            out.write(
                f"{dt_str}"
                f"\t{time_s:.3f}"
                f"\t{apqj:.4f}"
                f"\t{cptv:.3f}"
                f"\t{cptvc1:.3f}"
                f"\t{ptvc1:.3f}"
                f"\t{rtdqj:.3f}"
                f"\t{spep:.3f}\n"
            )

    print(f"Wrote TXT with datetime + named columns to: {out_path}")

# Example usage:


las_file = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_S2_7in_WL_run31_01Nov2025_RCX_MICROFRAC_2291.0M.las")
out_txt  = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2291.0M_converted_simple.txt")

convert_delft_las_to_txt_simple(las_file, out_txt)
print("Written:", out_txt)

Wrote TXT with datetime + named columns to: Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2291.0M_converted_simple.txt
Written: Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2291.0M_converted_simple.txt


# XLOT 3

In [2]:
from pathlib import Path
from datetime import datetime, timedelta
import re


def convert_delft_las_to_txt_simple(las_path: Path, out_path: Path):
    """
    Convert Delft LAS file to a simple TXT with:
      Datetime, TIME [s], APQJ [bar], CPTV [cm3], CPTVC1 [cm3],
      PTVC1 [cm3], RTDQJ [degC], SPEP [psi]

    - Datetime is reconstructed from TLAB. (or DATE. fallback) + TIME seconds.
    - All values are kept in the units given in the LAS file.
    """
    las_path = Path(las_path)
    out_path = Path(out_path)

    with open(las_path, "r", errors="ignore") as f:
        lines = f.readlines()

    # ----------------------------------------------------
    # 1) Extract absolute start time t0 from TLAB. (preferred) or DATE.
    # ----------------------------------------------------
    t0 = None
    for line in lines:
        if line.strip().startswith("TLAB."):
            # Example: "TLAB.       16:00 01-NOV-2025  : Time Logger At Bottom"
            m = re.search(r"TLAB\.\s+([0-9:]+)\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
            if m:
                time_str, date_str = m.groups()
                t0 = datetime.strptime(f"{time_str} {date_str}", "%H:%M %d-%b-%Y")
                break

    if t0 is None:
        for line in lines:
            if line.strip().startswith("DATE."):
                # Example: "DATE.       01-NOV-2025 : Log Date"
                m = re.search(r"DATE\.\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
                if m:
                    date_str = m.group(1)
                    t0 = datetime.strptime(f"00:00 {date_str}", "%H:%M %d-%b-%Y")
                    break

    if t0 is None:
        # Very defensive fallback
        t0 = datetime(2025, 1, 1)

    # ----------------------------------------------------
    # 2) Locate the ~A block and parse curve names
    # ----------------------------------------------------
    a_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("~A"):
            a_idx = i
            break
    if a_idx is None:
        raise RuntimeError("No ~A block found in LAS file.")

    header_line = lines[a_idx].strip()
    curve_names = header_line.split()[1:]  # skip "~A"
    col_idx = {name: j for j, name in enumerate(curve_names)}

    # Ensure required curves exist
    required_curves = ["TIME", "APQJ", "CPTV", "CPTVC1", "PTVC1", "RTDQJ", "SPEP"]
    for rc in required_curves:
        if rc not in col_idx:
            raise RuntimeError(f"Required curve {rc} not found in LAS file.")

    ncols = len(curve_names)

    # ----------------------------------------------------
    # 3) Write TXT: datetime + named columns
    # ----------------------------------------------------
    with open(out_path, "w", encoding="utf-8") as out:
        # Single header row with clear names + units
        out.write(
            "Datetime"
            "\tTIME [s]"
            "\tAPQJ [bar]"
            "\tCPTV [cm3]"
            "\tCPTVC1 [cm3]"
            "\tPTVC1 [cm3]"
            "\tRTDQJ [degC]"
            "\tSPEP [psi]\n"
        )

        # Data lines
        for line in lines[a_idx + 1:]:
            if not line.strip():
                continue

            parts = line.strip().split()
            if len(parts) != ncols:
                continue  # malformed / end-of-data

            try:
                time_s = float(parts[col_idx["TIME"]])
                apqj   = float(parts[col_idx["APQJ"]])
                cptv   = float(parts[col_idx["CPTV"]])
                cptvc1 = float(parts[col_idx["CPTVC1"]])
                ptvc1  = float(parts[col_idx["PTVC1"]])
                rtdqj  = float(parts[col_idx["RTDQJ"]])
                spep   = float(parts[col_idx["SPEP"]])
            except ValueError:
                # skip lines that can't be parsed numerically
                continue

            # Build datetime from t0 + TIME [s]
            timestamp = t0 + timedelta(seconds=time_s)
            dt_str = timestamp.strftime("%d-%m-%Y %H:%M:%S")

            out.write(
                f"{dt_str}"
                f"\t{time_s:.3f}"
                f"\t{apqj:.4f}"
                f"\t{cptv:.3f}"
                f"\t{cptvc1:.3f}"
                f"\t{ptvc1:.3f}"
                f"\t{rtdqj:.3f}"
                f"\t{spep:.3f}\n"
            )

    print(f"Wrote TXT with datetime + named columns to: {out_path}")

# Example usage:


las_file = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_S2_7in_WL_run31_01Nov2025_RCX_MICROFRAC_2365.1M.las")
out_txt  = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2365.1M_converted_simple.txt")

convert_delft_las_to_txt_simple(las_file, out_txt)
print("Written:", out_txt)

Wrote TXT with datetime + named columns to: Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2365.1M_converted_simple.txt
Written: Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2365.1M_converted_simple.txt


# XLOT 4

In [3]:
from pathlib import Path
from datetime import datetime, timedelta
import re


def convert_delft_las_to_txt_simple(las_path: Path, out_path: Path):
    """
    Convert Delft LAS file to a simple TXT with:
      Datetime, TIME [s], APQJ [bar], CPTV [cm3], CPTVC1 [cm3],
      PTVC1 [cm3], RTDQJ [degC], SPEP [psi]

    - Datetime is reconstructed from TLAB. (or DATE. fallback) + TIME seconds.
    - All values are kept in the units given in the LAS file.
    """
    las_path = Path(las_path)
    out_path = Path(out_path)

    with open(las_path, "r", errors="ignore") as f:
        lines = f.readlines()

    # ----------------------------------------------------
    # 1) Extract absolute start time t0 from TLAB. (preferred) or DATE.
    # ----------------------------------------------------
    t0 = None
    for line in lines:
        if line.strip().startswith("TLAB."):
            # Example: "TLAB.       16:00 01-NOV-2025  : Time Logger At Bottom"
            m = re.search(r"TLAB\.\s+([0-9:]+)\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
            if m:
                time_str, date_str = m.groups()
                t0 = datetime.strptime(f"{time_str} {date_str}", "%H:%M %d-%b-%Y")
                break

    if t0 is None:
        for line in lines:
            if line.strip().startswith("DATE."):
                # Example: "DATE.       01-NOV-2025 : Log Date"
                m = re.search(r"DATE\.\s+(\d{1,2}-[A-Z]{3}-\d{4})", line)
                if m:
                    date_str = m.group(1)
                    t0 = datetime.strptime(f"00:00 {date_str}", "%H:%M %d-%b-%Y")
                    break

    if t0 is None:
        # Very defensive fallback
        t0 = datetime(2025, 1, 1)

    # ----------------------------------------------------
    # 2) Locate the ~A block and parse curve names
    # ----------------------------------------------------
    a_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("~A"):
            a_idx = i
            break
    if a_idx is None:
        raise RuntimeError("No ~A block found in LAS file.")

    header_line = lines[a_idx].strip()
    curve_names = header_line.split()[1:]  # skip "~A"
    col_idx = {name: j for j, name in enumerate(curve_names)}

    # Ensure required curves exist
    required_curves = ["TIME", "APQJ", "CPTV", "CPTVC1", "PTVC1", "RTDQJ", "SPEP"]
    for rc in required_curves:
        if rc not in col_idx:
            raise RuntimeError(f"Required curve {rc} not found in LAS file.")

    ncols = len(curve_names)

    # ----------------------------------------------------
    # 3) Write TXT: datetime + named columns
    # ----------------------------------------------------
    with open(out_path, "w", encoding="utf-8") as out:
        # Single header row with clear names + units
        out.write(
            "Datetime"
            "\tTIME [s]"
            "\tAPQJ [bar]"
            "\tCPTV [cm3]"
            "\tCPTVC1 [cm3]"
            "\tPTVC1 [cm3]"
            "\tRTDQJ [degC]"
            "\tSPEP [psi]\n"
        )

        # Data lines
        for line in lines[a_idx + 1:]:
            if not line.strip():
                continue

            parts = line.strip().split()
            if len(parts) != ncols:
                continue  # malformed / end-of-data

            try:
                time_s = float(parts[col_idx["TIME"]])
                apqj   = float(parts[col_idx["APQJ"]])
                cptv   = float(parts[col_idx["CPTV"]])
                cptvc1 = float(parts[col_idx["CPTVC1"]])
                ptvc1  = float(parts[col_idx["PTVC1"]])
                rtdqj  = float(parts[col_idx["RTDQJ"]])
                spep   = float(parts[col_idx["SPEP"]])
            except ValueError:
                # skip lines that can't be parsed numerically
                continue

            # Build datetime from t0 + TIME [s]
            timestamp = t0 + timedelta(seconds=time_s)
            dt_str = timestamp.strftime("%d-%m-%Y %H:%M:%S")

            out.write(
                f"{dt_str}"
                f"\t{time_s:.3f}"
                f"\t{apqj:.4f}"
                f"\t{cptv:.3f}"
                f"\t{cptvc1:.3f}"
                f"\t{ptvc1:.3f}"
                f"\t{rtdqj:.3f}"
                f"\t{spep:.3f}\n"
            )

    print(f"Wrote TXT with datetime + named columns to: {out_path}")

# Example usage:


las_file = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_S2_7in_WL_run31_01Nov2025_RCX_MICROFRAC_2506_7M.las")
out_txt  = Path("Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2506_7M_converted_simple.txt")

convert_delft_las_to_txt_simple(las_file, out_txt)
print("Written:", out_txt)

Wrote TXT with datetime + named columns to: Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2506_7M_converted_simple.txt
Written: Data/XLOT_minifrac/Delft/TU_DELFT_GT02_MICROFRAC_2506_7M_converted_simple.txt
